# Tests

## Block tests

In [1]:
from residual_blocks import ClassicBottleneck, InvertedBottleneck, ConvNeXtBlock
import torch
from architecture import Stem, Body, Head

### Classic bottleneck block

In [2]:
classic_bottleneck_block = ClassicBottleneck(10,5,3)

c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
# Batch, exterior channels, x, y
test_input = torch.rand([3,10,50,50])
classic_bottleneck_block(test_input).size()

torch.Size([3, 10, 50, 50])

### Inverse bottleneck block

In [4]:
inverted_bottleneck_block = InvertedBottleneck(10,10)

In [5]:
test_input = torch.rand([3,10,50,50])
inverted_bottleneck_block(test_input).size()

torch.Size([3, 10, 50, 50])

In [6]:
resid_output = inverted_bottleneck_block(test_input)
inverted_bottleneck_block.use_residual = False
resid_output_2 = inverted_bottleneck_block(test_input) + test_input

all((resid_output == resid_output_2).reshape(-1))

True

In [7]:
inverted_bottleneck_block_2 = InvertedBottleneck(10,20, kernel_size=7,stride=4, use_residual=False)

In [8]:
inverted_bottleneck_block_2(test_input).size()

torch.Size([3, 20, 13, 13])

### ConvNeXt

In [9]:
convnext_block = ConvNeXtBlock(10,10)

In [10]:
convnext_block(test_input).size()

torch.Size([3, 10, 50, 50])

## Parameter Counts

In [11]:
def count_params(model):
    return sum(p.numel() for p in model.parameters())

In [12]:
fake_input = torch.randn((2,3,224,224)) #batch_size 2, 3 input channels
stem = Stem(out_channels=80, stem_type='patchify')
stem(fake_input)
print(count_params(stem))

4080


In [13]:
stem(fake_input).size()

torch.Size([2, 80, 56, 56])

In [14]:
body = Body()
out = body(stem(fake_input))
count_params(body)

14665920

In [15]:
out.size()

torch.Size([2, 640, 7, 7])

In [16]:
pooltest = torch.nn.AdaptiveAvgPool2d(1)

In [17]:
pooltest(out).size(1)

640

In [18]:
torch.nn.Flatten()(pooltest(out)).size()

torch.Size([2, 640])

In [19]:
head = Head(out.size(1))

In [20]:
count_params(head)

4487641

In [21]:
count_params(body)+count_params(head)+count_params(stem)

19157641

In [22]:
head(out)

tensor([[-0.0079, -0.0170, -0.0342,  ...,  0.1197, -0.0084, -0.0771],
        [ 0.0786, -0.0225, -0.0203,  ..., -0.0685, -0.0569,  0.0127]],
       grad_fn=<AddmmBackward0>)